In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

file_name = '../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv'

In [ ]:
 import datetime
import os

#define a conversion function for the native timestamps in the csv file
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))

# read in the data and apply our conversion function, this spits out a DataFrame with the DateTimeIndex already in place
print('Using bitstampUSD_1-min_data...')
data = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv', parse_dates=True, date_parser=dateparse, index_col=[0])

print('Total null open prices: %s' % data['Open'].isnull().sum())

In [ ]:
!pip install stocktrends
import stocktrends as st

In [ ]:
# First thing is to fix the data for bars/candles where there are no trades. 
# Volume/trades are a single event so fill na's with zeroes for relevant fields...
data['Volume_(BTC)'].fillna(value=0, inplace=True)
data['Volume_(Currency)'].fillna(value=0, inplace=True)
data['Weighted_Price'].fillna(value=0, inplace=True)

# next we need to fix the OHLC (open high low close) data which is a continuous timeseries so
# lets fill forwards those values...
data['Open'].fillna(method='ffill', inplace=True)
data['High'].fillna(method='ffill', inplace=True)
data['Low'].fillna(method='ffill', inplace=True)
data['Close'].fillna(method='ffill', inplace=True)

# check how we are looking now, should be nice and clean...
print(data)

In [ ]:
# The first thing we need are our trading signals. The Turtle strategy was based on daily data and
# they used to enter breakouts (new higher highs or new lower lows) in the 22-60 day range roughly.
# We are dealing with minute bars here so a 22 minute new high isn't much to get excited about. Lets
# pick an equivalent to 60 days then. They also only considered Close price so lets do the same...

signal_lookback = 60 * 24 * 60 # days * hours * minutes

# here's our signal columns
data['Buy'] = np.zeros(len(data))
data['Sell'] = np.zeros(len(data))

# this is our 'working out', you could collapse these into the .loc call later on and save memory 
# but I've left them in for debug purposes, makes it easier to see what is going on
data['RollingMax'] = data['Close'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).max()
data['RollingMin'] = data['Close'].shift(1).rolling(signal_lookback, min_periods=signal_lookback).min()
data.loc[data['RollingMax'] < data['Close'], 'Buy'] = 1
data.loc[data['RollingMin'] > data['Close'], 'Sell'] = -1

# lets now take a look and see if its doing something sensible
import matplotlib
import matplotlib.pyplot as plt

fig,ax1 = plt.subplots(1,1)
ax1.plot(data['Close'])
y = ax1.get_ylim()
ax1.set_ylim(y[0] - (y[1]-y[0])*0.4, y[1])

ax2 = ax1.twinx()
ax2.set_position(matplotlib.transforms.Bbox([[0.125,0.1],[0.9,0.32]]))
ax2.plot(data['Buy'], color='#77dd77')
ax2.plot(data['Sell'], color='#dd4444')


In [ ]:
!pip install yfinance
!pip install fix-yahoo-finance

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [ ]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data
finalData={} # This should contain our final output and that is Renko OHLC data
n=7 # Period for ATR
renkoData={} # It contains information on the lastest bar of renko data for the number of stocks we are working on

In [ ]:
def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.ix[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas version- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return

In [ ]:
StockList=['INFY', 'ICICIBANK']

In [ ]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

In [ ]:
for stock in data:
    if data[stock].empty:
        print(stock)

In [ ]:
for stock in data:
    print (stock, data[stock].head())

In [ ]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    ATR(data[stock],n)
    data[stock]=data[stock][['Open','High','Low','Close','ATR']] # Removing unwanted columns

In [ ]:
data['INFY'].head() #Since ATR period is 7, we will see atr column filling out from 7th row

In [ ]:
data['ICICIBANK'].head(10)

In [ ]:
for stock in data:
    renkoData[stock]={'BrickSize':0.0, 'Open':0.0,'Close':0.0,'Color':''}

In [ ]:
for stock in data:
    renkoData[stock]['BrickSize']=round(data[stock]['ATR'][-1],2) #This can be set manually as well!
    renkoData[stock]['Open']=renkoData[stock]['BrickSize']+renkoData[stock]['Close'] # This can be done the otherway round
                                                                                    # as well.'Close' = 'BrickSize' - 'Open' 
    renkoData[stock]['Color']='red'    # Should you choose to do the other way round, please change the color to 'green'

In [ ]:
renkoData['ICICIBANK']

In [ ]:
renkoData['INFY']

In [ ]:
for stock in data:
    finalData[stock]=pd.DataFrame()
    finalData[stock].index.name='Date'
    finalData[stock]['ReOpen']=0.0
    finalData[stock]['ReHigh']=0.0
    finalData[stock]['ReLow']=0.0
    finalData[stock]['ReClose']=0.0
    finalData[stock]['Color']=''

In [ ]:
for stock in data: # This loops thorugh all the stocks in the data dictionary
    for index,row in data[stock].iterrows(): # One may choose to use Pure python instead of Iterrows to loop though each n 
                                         # every row to improve performace if datasets are large.
        if renkoData[stock]['Open']> renkoData[stock]['Close']: 
            while row['Close']> renkoData[stock]['Open']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='green'

            while row['Close'] < renkoData[stock]['Close']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='red'
                
        else:
            while row['Close']< renkoData[stock]['Open']-renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']-=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']-=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Close']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['Color'].loc[index]='red'
                
            while row['Close'] > renkoData[stock]['Close']+renkoData[stock]['BrickSize']:
                renkoData[stock]['Open']+=renkoData[stock]['BrickSize']
                renkoData[stock]['Close']+=renkoData[stock]['BrickSize']
                finalData[stock].loc[index]=row
                finalData[stock]['ReOpen'].loc[index]= renkoData[stock]['Open']         
                finalData[stock]['ReHigh'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['ReLow'].loc[index]=renkoData[stock]['Open']
                finalData[stock]['ReClose'].loc[index]=renkoData[stock]['Close']
                finalData[stock]['Color'].loc[index]='green'

In [ ]:
finalData['ICICIBANK'].tail()

In [ ]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()

writer = pd.ExcelWriter('RenkoData.xlsx')
for stock in finalData:
    finalData[stock].to_excel(writer,stock)
writer.save()

In [ ]:
import socket, numpy as np
from sklearn.linear_model import LinearRegression


In [ ]:
class socketserver:
    def __init__(self, address = '', port = 9090):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.address = address
        self.port = port
        self.sock.bind((self.address, self.port))
        self.cummdata = ''
        
    def recvmsg(self):
        self.sock.listen(1)
        self.conn, self.addr = self.sock.accept()
        print('connected to', self.addr)
        self.cummdata = ''

        while True:
            data = self.conn.recv(10000)
            self.cummdata+=data.decode("utf-8")
            if not data:
                break    
            self.conn.send(bytes(calcregr(self.cummdata), "utf-8"))
            return self.cummdata
            
    def __del__(self):
        self.sock.close()

In [ ]:
def calcregr(msg = ''):
    chartdata = np.fromstring(msg, dtype=float, sep= ' ') 
    Y = np.array(chartdata).reshape(-1,1)
    X = np.array(np.arange(len(chartdata))).reshape(-1,1)
        
    lr = LinearRegression()
    lr.fit(X, Y)
    Y_pred = lr.predict(X)
    type(Y_pred)
    P = Y_pred.astype(str).item(-1) + ' ' + Y_pred.astype(str).item(0)
    print(P)
    return str(P)

In [ ]:
serv = socketserver('127.0.0.1', 9090)

while True:  
    msg = serv.recvmsg()

In [ ]:
bool socksend(int sock,string request) 
  {
   char req[];
   int  len=StringToCharArray(request,req)-1;
   if(len<0) return(false);
   return(SocketSend(sock,req,len)==len); 
  }

string socketreceive(int sock,int timeout)
  {
   char rsp[];
   string result="";
   uint len;
   uint timeout_check=GetTickCount()+timeout;
   do
     {
      len=SocketIsReadable(sock);
      if(len)
        {
         int rsp_len;
         rsp_len=SocketRead(sock,rsp,len,timeout);
         if(rsp_len>0) 
           {
            result+=CharArrayToString(rsp,0,rsp_len); 
           }
        }
     }
   while((GetTickCount()<timeout_check) && !IsStopped());
   return result;
  }
void drawlr(string points) 
  {
   string res[];
   StringSplit(points,' ',res);

   if(ArraySize(res)==2) 
     {
      Print(StringToDouble(res[0]));
      Print(StringToDouble(res[1]));
      datetime temp[];
      CopyTime(Symbol(),Period(),TimeCurrent(),lrlenght,temp);
      ObjectCreate(0,"regrline",OBJ_TREND,0,TimeCurrent(),NormalizeDouble(StringToDouble(res[0]),_Digits),temp[0],NormalizeDouble(StringToDouble(res[1]),_Digits)); 
     }


In [ ]:
void OnTick() {
 socket=SocketCreate();
 if(socket!=INVALID_HANDLE) {
  if(SocketConnect(socket,"localhost",9090,1000)) {
   Print("Connected to "," localhost",":",9090);
         
   double clpr[];
   int copyed = CopyClose(_Symbol,PERIOD_CURRENT,0,lrlenght,clpr);
         
   string tosend;
   for(int i=0;i<ArraySize(clpr);i++) tosend+=(string)clpr[i]+" ";       
   string received = socksend(socket, tosend) ? socketreceive(socket, 10) : ""; 
   drawlr(recieved); }
   
  else Print("Connection ","localhost",":",9090," error ",GetLastError());
  SocketClose(socket); }
 else Print("Socket creation error ",GetLastError()); }

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import metatrader as mt5
 
# подключимся к MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
 
# запросим статус и параметры подключения
print(mt5.terminal_info())
# получим информацию о версии MetaTrader 5
print(mt5.version())
 
# запросим 1000 тиков с EURAUD 
euraud_ticks = mt5.copy_ticks_from("EURAUD", datetime(2020,1,28,13), 1000, mt5.COPY_TICKS_ALL)
# запросим тики с AUDUSD в интервале 2019.04.01 13:00 - 2019.04.02 13:00 
audusd_ticks = mt5.copy_ticks_range("AUDUSD", datetime(2020,1,27,13), datetime(2020,1,28,13), mt5.COPY_TICKS_ALL)
 
# получим бары с разных инструментов разными способами
eurusd_rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M1, datetime(2020,1,28,13), 1000)
eurgbp_rates = mt5.copy_rates_from_pos("EURGBP", mt5.TIMEFRAME_M1, 0, 1000)
eurcad_rates = mt5.copy_rates_range("EURCAD", mt5.TIMEFRAME_M1, datetime(2020,1,27,13), datetime(2020,1,28,13))
 
# завершим подключение к MetaTrader 5
mt5.shutdown()
 
#DATA
print('euraud_ticks(', len(euraud_ticks), ')')
for val in euraud_ticks[:10]: print(val)
 
print('audusd_ticks(', len(audusd_ticks), ')')
for val in audusd_ticks[:10]: print(val)
 
print('eurusd_rates(', len(eurusd_rates), ')')
for val in eurusd_rates[:10]: print(val)
 
print('eurgbp_rates(', len(eurgbp_rates), ')')
for val in eurgbp_rates[:10]: print(val)
 
print('eurcad_rates(', len(eurcad_rates), ')')
for val in eurcad_rates[:10]: print(val)
 
#PLOT
# создадим из полученных данных DataFrame
ticks_frame = pd.DataFrame(euraud_ticks)
# сделаем отрисовку тиков на графике
plt.plot(ticks_frame['time'], ticks_frame['ask'], 'r-', label='ask')
plt.plot(ticks_frame['time'], ticks_frame['bid'], 'b-', label='bid')
 
# выведем легенды 
plt.legend(loc='upper left')
 
# добавим заголовок 
plt.title('EURAUD ticks')
 
# покажем график
plt.show()